In [8]:
import shutil
import os

folders_to_delete = [
    "Language_Model",
    "Language Model",
    "miniproject2_language_model.ipynb"
]

for folder in folders_to_delete:
    if os.path.exists(folder):
        if os.path.isfile(folder):
            os.remove(folder)
            print(f"Deleted file: {folder}")
        else:
            shutil.rmtree(folder)
            print(f"Deleted folder: {folder}")
    else:
        print(f"Not found: {folder}")

print("All target folders cleared!")
!ls -a

Deleted folder: Language_Model
Not found: Language Model
Not found: miniproject2_language_model.ipynb
All target folders cleared!
.  ..  .config	sample_data


In [9]:
# clone project
!git clone https://github.com/JennyGVoice/Language_Model.git

Cloning into 'Language_Model'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 56 (delta 20), reused 48 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 438.69 KiB | 13.29 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [10]:
%cd Language_Model
!pwd
!ls -a

/content/Language_Model
/content/Language_Model
.   experiments  main.ipynb			    requirements.txt
..  .git	 miniproject2_language_model.ipynb  src


In [11]:
# test cuda availability
import torch
torch.cuda.is_available(), torch.cuda.get_device_name()

(True, 'Tesla T4')

In [12]:
!nvidia-smi

Tue Dec  9 19:55:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
%pip install -r requirements.txt

In [ ]:
from src.train import main
main()

Using device: cuda


In [ ]:
train_dataset = CharDataset(text, block_size=128, split="train")
x, y = train_dataset[0]
print(x.shape, y.shape)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [ ]:
import torch
from src.dataset import CharDataset
from src.model.gpt_like import GPTLanguageModel

# load data
with open("data/input.txt", "r") as f:
    text = f.read()

dataset = CharDataset(text, block_size=128, split="train")

# choose checkpoint
ckpt_path = "experiments/checkpoints/step_200.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = GPTLanguageModel(dataset.vocab_size).to(device)

# load checkpoint
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()

# test generation
prompt = "O God, O God!"
idx = torch.tensor([dataset.encode(prompt)]).to(device)
generated = model.generate(idx, max_new_tokens=300)
print(dataset.decode(generated[0].tolist()))

In [ ]:
from torch.utils.data import DataLoader
from src.model.metrics import calculate_ppl

val_dataset = CharDataset(text, block_size=128, split="val")
val_loader = DataLoader(val_dataset, batch_size=64)

ppl = calculate_ppl(model, val_loader, device=device)
print("Validation Perplexity:", ppl)